In [ ]:
# Drill-Funktionen importieren
import drill as d

In [ ]:
import requests
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as mplt
import plotly.express as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import datetime as dt
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.datasets import load_digits
from sklearn import metrics
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def encode_cyclical(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [ ]:
def create_plot(data):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['timestamp'], 
                             y=data['co2_ppm'], 
                             mode='markers',
                             marker=dict(
                                 color=(
                                    (data['presence'] == 0)).astype('int'),
                                    colorscale=[[0, 'yellow'], [1, 'blue']])))
    return fig

In [ ]:
#df = d.get_PIR_data()
#df.to_json('data.json')

In [ ]:
df = pd.read_json('data.json') 

In [ ]:
plt.scatter(df, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
df_test = df

df_nov = df.loc[(df['timestamp'] < pd.to_datetime(1636722000, unit='s', origin='unix')) &
                (df['timestamp'] > pd.to_datetime(1634562000, unit='s', origin='unix'))]

df_nov.reset_index(drop=True, inplace=True)
# alle zwischen 18.10. und 22.11. entfernen
df_test = df.drop(df[(df['timestamp'] > pd.to_datetime(1634518800, unit='s', origin='unix')) & 
                    (df['timestamp'] < pd.to_datetime(1637586000, unit='s', origin='unix'))].index)


# timestamp etwas leichter zu verarbeiten, wenn als Integer gespeichert
df_test = df_test.assign(hoursMinutesSeconds=lambda d: (d['timestamp'].dt.hour.astype('int') * 10000 + 
                                                        d['timestamp'].dt.minute.astype('int') * 100 + 
                                                       d['timestamp'].dt.second.astype('int')))

df_test['hour_sin'] = np.sin(2 * np.pi * df_test['hoursMinutesSeconds']/235959.0)
df_test['hour_cos'] = np.cos(2 * np.pi * df_test['hoursMinutesSeconds']/235959.0)

df_nov = df_nov.assign(hoursMinutesSeconds=lambda d: (d['timestamp'].dt.hour.astype('int') * 10000 + 
                                                        d['timestamp'].dt.minute.astype('int') * 100 + 
                                                       d['timestamp'].dt.second.astype('int')))

df_nov['hour_sin'] = np.sin(2 * np.pi * df_nov['hoursMinutesSeconds']/235959.0)
df_nov['hour_cos'] = np.cos(2 * np.pi * df_nov['hoursMinutesSeconds']/235959.0)

In [ ]:
plt.scatter(df_test, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# erhoeht sich die Genauigkeit, wenn Datenset ausgeglichener ist?
# Idee bei ca. 8 Stunden Anwesenheit:
# Entferne 8 Nachtstunden von 22:00-05:00, 
# sodass von 24 stunden 8 Anwesenheits-Stunden und 8 Abwesenheits-Stunden uebrig bleiben
df_test.drop(df_test[(df_test['hoursMinutesSeconds'] < 50000) |
                     (df_test['hoursMinutesSeconds'] > 220000)].index, inplace=True)

df_nov.drop(df_nov[(df_nov['hoursMinutesSeconds'] < 50000) |
                     (df_nov['hoursMinutesSeconds'] > 220000)].index, inplace=True)


In [ ]:
plt.scatter(df_nov, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# Template für mehrere Plots in einem Graphen
#fig = create_plot(df_test)
#df_test['co2_ppmShifted'] = df_test['co2_ppm'].shift(12)
#fig.add_trace(go.Scatter(x=df_test['timestamp'], 
#                             y=df_test['co2_ppmShifted'], 
#                             mode='lines',
#                             line=dict(width=2),
#                             marker=dict(
#                                 color=(
#                                    (df_test['presence'] == 0)).astype('int'),
#                                    colorscale=[[0, 'green'], [1, 'red']])))

#fig.show()
#plt.scatter(df_test, x='timestamp', y='co2_ppm', color='presence')

In [ ]:
# Steigungen mit Abstaenden 5, 30 und 60 Minuten einfuegen
df_test['co2_ppm_deltaOne'] = df_test['co2_ppm'] - df_test.shift(1)['co2_ppm']
df_test['co2_ppm_deltaSix'] = df_test['co2_ppm'] - df_test.shift(6)['co2_ppm']
df_test['co2_ppm_deltaTwelve'] = df_test['co2_ppm'] - df_test.shift(12)['co2_ppm']
#df_test['co2_ppm_deltaMinusOne'] = df_test.shift(-1)['co2_ppm'] - df_test['co2_ppm']
#df_test['co2_ppm_deltaMinusSix'] = df_test.shift(-6)['co2_ppm'] - df_test['co2_ppm']
#df_test['co2_ppm_deltaMinusTwelve'] = df_test.shift(-12)['co2_ppm'] - df_test['co2_ppm']

# Falsche November-Daten ebenfalls mit Delta versehen
df_nov['co2_ppm_deltaOne'] = df_nov['co2_ppm'] - df_nov.shift(1)['co2_ppm']
df_nov['co2_ppm_deltaSix'] = df_nov['co2_ppm'] - df_nov.shift(6)['co2_ppm']
df_nov['co2_ppm_deltaTwelve'] = df_nov['co2_ppm'] - df_nov.shift(12)['co2_ppm']

In [ ]:
# Wochentag einfuegen
# verringert Genauigkeit, weil wahrscheinlich zu "verlaesslich"

#df_test['dayOfWeek'] = df['timestamp'].dt.dayofweek
#df_test = df_test.drop(df_test[df_test.dayOfWeek > 4].index)

In [ ]:
# Ausreisser mit Interquartile Range (IQR) und Tukey's Method loeschen
# Idee: Fenster wird an fast allen Tagen ab ca. 1200-1300 ppm geoeffnet
# -> was veraendert sich, wenn die Werte darüber entfernt werden und das Datenset so "geglaettet" wird?
# Erhoeht Genauigkeit um ca. 2%
x = df_test['co2_ppm']
q1 = np.percentile(x, 12)
q3 = np.percentile(x, 88)
iqr = q3 - q1
floor = q1 - 1.5*iqr
ceiling = q3 + 1.5*iqr
outlier_indices = list(x.index[(x < floor) | (x > ceiling)])
outlier_values = list(x[outlier_indices])
print(outlier_values)

In [ ]:
df_test = df_test.drop(index=outlier_indices)

In [ ]:
# timestamp, presence, temperatur und humidity entfernen
# temp/humid erhoehen Genauigkeit deutlich, da relativ unverlaesslich -> von zu vielen aeusseren Faktoren abhaengig
df_test = df_test.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_timestamp = df_test['timestamp']
y_presence = df_test['presence']
X_presence = df_test.drop(['timestamp', 'hoursMinutesSeconds', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X_presence, y_presence, test_size=0.2, random_state=1, shuffle=False)

In [ ]:
# shift des trainings-sets um 5 minuten in die Vergangenheit
# -> Test ob Model auch in die Zukunft Erwartungen treffen kann
#ytrain = ytrain.shift(-6)
#ytrain.dropna(axis=0, how='any', inplace=False)
#ytrain = ytrain.replace(np.nan, 0)

In [ ]:
# Random Forest Classification
modelClass = RandomForestClassifier(n_estimators=250)
modelClass.fit(Xtrain, ytrain)
ypred = modelClass.predict(X_presence)
accuracy_score(y_presence, ypred)

In [ ]:
df_valid_class = df_test.copy()
df_valid_class['timestamp'] = df_timestamp
df_valid_class['prediction'] = ypred
df_valid_class['co2_ppm'] = df_test['co2_ppm']

plt.scatter(df_valid_class, x='timestamp', y='co2_ppm', color='prediction')

In [ ]:
# plot feature-importance
feature_imp = pd.Series(modelClass.feature_importances_,index=Xtrain.columns).sort_values(ascending=False)
# Creating a bar plot
ax = sns.barplot(x=feature_imp, y=feature_imp.index)
ax.set(xlabel='Importance', ylabel='Feature')

In [ ]:
# Model Cross-Validation with Random Forest Classifier
ytest_model = modelClass.fit(Xtrain, ytrain).predict(Xtest)
ytrain_model = modelClass.fit(Xtest, ytest).predict(Xtrain)
accuracy_score(ytrain, ytrain_model), accuracy_score(ytest, ytest_model)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(modelClass, X_presence, y_presence, cv=5)

In [ ]:
# Random Forest Regression
modelReg = RandomForestRegressor(n_estimators=250)
modelReg.fit(Xtrain, ytrain)

ypred = modelReg.predict(Xtest)

In [ ]:
df_valid = Xtest.copy()
df_valid['timestamp'] = df_timestamp
df_valid['prediction'] = ypred
df_valid['co2_ppm'] = df_test['co2_ppm']
df_valid['presence'] = ytest

In [ ]:
df_test.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_timestamp = df_test['timestamp']
df_test.drop(['timestamp', 'hoursMinutesSeconds', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1, inplace=True)
ypred = modelReg.predict(df_test)

df_valid_test = df_test.copy()
df_valid_test['timestamp'] = df_timestamp
df_valid_test['prediction'] = ypred
df_valid_test['co2_ppm'] = df_test['co2_ppm']

In [ ]:
plt.scatter(df_valid_test, x='timestamp', y='co2_ppm', color='prediction')

In [ ]:
# falsche Werte im November korrigieren
df_nov.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_timestamp_nov = df_nov['timestamp']
df_nov.drop(['timestamp', 'hoursMinutesSeconds', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1, inplace=True)
ypred_nov = modelReg.predict(df_nov)

df_valid_nov = df_nov.copy()
df_valid_nov['timestamp'] = df_timestamp_nov
df_valid_nov['prediction'] = ypred_nov
df_valid_nov['co2_ppm'] = df_nov['co2_ppm']

df_valid_nov.loc[df_valid_nov['prediction'] >= 0.5, 'prediction'] = 1
df_valid_nov.loc[df_valid_nov['prediction'] < 0.5, 'prediction'] = 0


In [ ]:
plt.scatter(df_valid_nov, x='timestamp', y='co2_ppm', color='prediction')

In [ ]:
# Parameter tuning for Random Forest Models
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_reg = modelReg
rf_random_reg = RandomizedSearchCV(estimator = rf_reg, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random_reg.fit(Xtrain, ytrain)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_class = modelClass
rf_random_class = RandomizedSearchCV(estimator = rf_class, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random_class.fit(Xtrain, ytrain)

In [ ]:
rf_random_reg.best_params_

In [ ]:
rf_random_class.best_params_

In [ ]:
def evaluateClassifier(model, test_features, test_labels):
    ypred = model.predict(test_features)
    accuracy = accuracy_score(test_labels, ypred)
    print('Model Performance')
    print('Accuracy = {:0.2f}%.'.format(accuracy*100))
    
    return accuracy

In [ ]:
def evaluateRegressor(model, test_features, test_labels):
    ypred = modelReg.predict(test_features)
    errors = abs(ypred - test_labels)
    mse = mean_squared_error(test_labels, ypred)
    rmse = sqrt(mse) * 100
    accuracy = 100 - rmse
    print('Model Performance')
    print('Average Error: {:0.2f} degrees.'.format(np.mean(errors)*100))
    print('Accuracy for Random Forest: {:0.2f}%.'.format(accuracy)) 
    
    return accuracy

In [ ]:
base_model = modelClass
base_model.fit(Xtrain, ytrain)
base_accuracy = evaluateClassifier(base_model, Xtest, ytest)

best_random = rf_random_class.best_estimator_
random_accuracy = evaluateClassifier(best_random, Xtest, ytest)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

In [ ]:
#base_model = RandomForestRegressor(n_estimators = 250, random_state = 42)
base_model = modelReg
base_model.fit(Xtrain, ytrain)
base_accuracy = evaluateRegressor(base_model, Xtest, ytest)

best_random = rf_random_reg.best_estimator_
random_accuracy = evaluateRegressor(best_random, Xtest, ytest)

print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))